# Intro

This is really just a practice project, messing around with SQL, Pandas, and some of the data-wrangling techniques from my IBM 'Data Analysis with Python' course. The 'autos.csv' file was a dataset provided for 'Databases with SQL and Python' project previously. The dataset is at https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data. 

# Preparations

In [29]:
# Import the packages
import csv, sqlite3 
import pandas as pd
import numpy as np
!pip install ipython-sql

# Establish the SQL database
con = sqlite3.connect("autodb.db")
cur = con.cursor()
# Connect to it
%load_ext sql
%sql sqlite:///autodb.db
# Load the data from auto.csv
df = pd.read_csv("auto.csv")
df.to_sql("AUTO",con,if_exists='replace',index=False,method="multi")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


204

Now that the database is prepared, I'll quickly test to see if everything is loaded up properly. I realize this is perhaps a bit redundant, but I'm really just trying to practice my SQL stuff now that I've finished that course.

In [14]:
%%sql
SELECT * FROM AUTO LIMIT 5;

 * sqlite:///autodb.db
Done.


3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
2,?,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


## Cleaning

Okay, so I can immediately see some issues. First problem is the fact that there's no actual header, second is that I'm going to need to go through and figure out how to deal with those missing values. As I'm going along here, I'm realizing that really, most of my work is going to be done in Pandas, so with regards to the SQL aspect, I think I'll focus on just creating the database with the cleaned data at the end.

In [52]:
# Assign headers to a list of strings
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]
df.columns = headers # Update the underlying dataframe
df.to_sql("AUTO",con,if_exists='replace',index=False,method="multi") # Overwrite the data in the database.
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,15250.0


Next, I'm going to determine the number of missing values in each column.

In [33]:
# Replace "?" with NA
df.replace('?',np.nan,inplace=True)
df.isna().sum()

symboling             0
normalized-losses    40
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

So, we can see that 40 of the normalized loss values are missing. That just will not do. We don't want to drop 40 rows from our data-set, so we'll find the mean of the remaining values and substitute it in

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          204 non-null    int64  
 1   normalized-losses  164 non-null    object 
 2   make               204 non-null    object 
 3   fuel-type          204 non-null    object 
 4   aspiration         204 non-null    object 
 5   num-of-doors       202 non-null    object 
 6   body-style         204 non-null    object 
 7   drive-wheels       204 non-null    object 
 8   engine-location    204 non-null    object 
 9   wheel-base         204 non-null    float64
 10  length             204 non-null    float64
 11  width              204 non-null    float64
 12  height             204 non-null    float64
 13  curb-weight        204 non-null    int64  
 14  engine-type        204 non-null    object 
 15  num-of-cylinders   204 non-null    object 
 16  engine-size        204 non

## Converting Types

I'll have to change a lot of these columns to the proper type.

In [45]:
pd.set_option('display.max_columns', None) 
df.head() 

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [46]:
# Create a list of columns to change the type of in a simple way
cols = ["normalized-losses","bore","stroke","horsepower","peak-rpm","price"]

In [48]:
for col in cols:
    df[col] = df[col].astype("float64")

Now, another problem I can see here is the fact that "num-of-cylinders" is written as a bunch of strings, even though it should be an integer. Time to fix that...

In [63]:
%%sql
SELECT DISTINCT `num-of-cylinders` FROM AUTO

 * sqlite:///autodb.db
Done.


num-of-cylinders
four
six
five
three
twelve
two
eight


(Confession: I can't remember off the top of my head how to find the unique entries in a pandas series, hence why I used the SQL query)
Now I'll create a dictionary with each 'num-of-cylinders' value as the key and the equivalent numerical value as the value, then use a "for" loop to iterate through and replace them all

In [69]:
# Create the dictionary
dict = {'four':4,'six':6,'five':5,'three':3,'twelve':12,'two':2,'eight':8}
# Loop through the data frame and replace
for key,value in dict.items():
    df["num-of-cylinders"] = df["num-of-cylinders"].replace(key,value)
df["num-of-cylinders"] = df["num-of-cylinders"].astype("float64")

In [70]:
df.head() 

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,4.0,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,6.0,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,4.0,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,5.0,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,5.0,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,15250.0


Nice! Now, I'll end my work for now by updating the SQL database.

In [71]:
df.to_sql("AUTO",con,if_exists='replace',index=False,method="multi")
%sql SELECT * FROM AUTO

 * sqlite:///autodb.db
Done.


symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,None,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,4.0,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
1,None,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,6.0,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,4.0,109,mpfi,3.19,3.4,10.0,102.0,5500.0,24,30,13950.0
2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,5.0,136,mpfi,3.19,3.4,8.0,115.0,5500.0,18,22,17450.0
2,None,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,5.0,136,mpfi,3.19,3.4,8.5,110.0,5500.0,19,25,15250.0
1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,5.0,136,mpfi,3.19,3.4,8.5,110.0,5500.0,19,25,17710.0
1,None,audi,gas,std,four,wagon,fwd,front,105.8,192.7,71.4,55.7,2954,ohc,5.0,136,mpfi,3.19,3.4,8.5,110.0,5500.0,19,25,18920.0
1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,5.0,131,mpfi,3.13,3.4,8.3,140.0,5500.0,17,20,23875.0
0,None,audi,gas,turbo,two,hatchback,4wd,front,99.5,178.2,67.9,52.0,3053,ohc,5.0,131,mpfi,3.13,3.4,7.0,160.0,5500.0,16,22,None
2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,4.0,108,mpfi,3.5,2.8,8.8,101.0,5800.0,23,29,16430.0
